In [2]:
import json
import pandas as pd
import tensorflow as tf
from datetime import datetime as dt
tf.config.run_functions_eagerly(True)
from hmc.utils.dir import create_dir

In [3]:
from tqdm.notebook import tqdm

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
import os
from datetime import datetime

def horario_atual_para_diretorio():
    # Obtém a data e hora atual
    data_hora_atual = datetime.now()
    
    # Formata a data e hora para o formato de diretório desejado
    diretorio = data_hora_atual.strftime('%Y_%m_%d_%H_%M_%S')
    
    return diretorio

# Exemplo de uso
diretorio_formatado = horario_atual_para_diretorio()

In [6]:
# Set python level verbosity
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.DEBUG)

# Set C++ Graph Execution level verbosity
os.environ['TF_CPP_MIN_LOG_LEVEL'] = str(tf.compat.v1.logging.DEBUG)

base_path = "/mnt/disks/data/fma/trains"
sample_id = "hierarchical_tworoots_dev"


train_path = os.path.join(base_path, sample_id)
tfrecords_path =os.path.join(train_path,'tfrecords')
metadata_path = os.path.join(train_path,"metadata.json")
labels_path = os.path.join(train_path,"labels.json")

model_id = diretorio_formatado

model_path = os.path.join(train_path, model_id)

args = pd.Series({
    "batch_size":32,
    "epochs":10,
    "dropout":0.7,
    'patience':1,
    'max_queue_size':64,
    "labels_path": labels_path,
    "metadata_path": metadata_path,
    "trainset_pattern": os.path.join(tfrecords_path,'train'),
    "testset_pattern": os.path.join(tfrecords_path,'test'),
    "valset_pattern": os.path.join(tfrecords_path,'val'),
    "model_path":model_path
})


In [7]:
create_dir(model_path)

True

In [8]:


BUFFER_SIZE = 10


class Dataset:
    def __init__(self,files,epochs, batch_size, levels_size):
        self.epochs = epochs
        self.batch_size = batch_size
        self.files = files
        self.depth = len(levels_size)
        self.levels_size = levels_size

    def load_dataframe(self, dataset):
        columns = ['features','labels']
        
        
        df = pd.DataFrame(
            dataset.as_numpy_iterator(),
            columns=columns
        )
    
        df.dropna(inplace=True)
    
        return df

    def build(self,df=False):
        
        files = [os.path.join(self.files,file) for file in os.listdir(self.files)]

        ds = tf.data.TFRecordDataset(files)
        
        '''''
            Shuffle and reapeat
        '''''
        
        #ds = ds.shuffle(buffer_size=1024 * 50 * 10)
        #ds = ds.repeat(count=self.epochs)
        
        
        '''''
            Map and batch
        '''''
        
                      
        ds = ds.map(self.__parse__, num_parallel_calls=None)

        if df==True:
            return self.load_dataframe(ds)
        
        ds = ds.batch(self.batch_size,drop_remainder=False)
        
        
                      
        ds = ds.prefetch(buffer_size=5)
        
        
        return ds

    def convert_to_binary(self, label, num_classes):
        # Filtrar índices negativos
        valid_indices = tf.boolean_mask(label, label >= 0)
        
        # Inicializar vetor binário
        binary_label = tf.zeros(num_classes, dtype=tf.float32)
        
        # Atualizar vetor binário com base nos rótulos válidos
        indices = tf.expand_dims(valid_indices, 1)
        updates = tf.ones_like(valid_indices, dtype=tf.float32)
        binary_label = tf.tensor_scatter_nd_update(binary_label, indices, updates)

        return binary_label

    def __parse__(self, element):
        data = {}
        for level in range(1, self.depth+1):
            data[f'label{level}'] = tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True)
        
        data.update({
            'features': tf.io.FixedLenFeature([1280], tf.float32),
            'track_id' : tf.io.FixedLenFeature([], tf.int64),
        })
        
        content = tf.io.parse_single_example(element, data)

        labels = {}
        for level in range(1, self.depth+1):
            local_label = content[f'label{level}']
            binary_label = self.convert_to_binary(local_label, self.levels_size[f'level{level}'])
            labels.update({f'level{level}': binary_label})
        

        inp = {"features":content['features'] }

        return inp, labels


In [9]:
'''
from keras import layers
from keras.layers import Dense, Dropout, Reshape, Concatenate, Flatten, Input, Normalization
from keras.optimizers import Adam
import tensorflow as tf

import numpy as np

# class OutputNormalization(layers.Layer):
#     def call(self, x, **kwargs):
#         norm_output = tf.one_hot(tf.math.argmax(x, axis=1), x.shape[1], dtype=x.dtype)
#         norm_output = Flatten()(norm_output)
#         norm_output = Reshape((1,))(norm_output)
#         return norm_output

#     def compute_output_shape(self, input_shape):
#         return input_shape

class OutputNormalization(layers.Layer):
    def call(self, x, **kwargs):
        return tf.one_hot(tf.math.argmax(x, axis=1), x.shape[1], dtype=x.dtype)

    def compute_output_shape(self, input_shape):
        return input_shape



def build_cnn(feature, input_shape):
    x: object = Normalization(input_shape=[input_shape, 1], axis=None)(feature)
    x = layers.Conv1D(128, 3, activation='relu', padding="valid")(x)
    x = layers.MaxPooling1D()(x)
    x = layers.Conv1D(64, 3, activation='relu', padding="valid")(x)
    x = layers.MaxPooling1D()(x)
    x = layers.Conv1D(32, 3, activation='relu', padding="valid")(x)
    x = layers.MaxPooling1D()(x)
    x = layers.Flatten()(x)

    return x


def build_classification(x, size, dropout, input_shape=1024, name='level1'):
    #x: object = Dense(input_shape, activation='relu')(x)
    #x = Dropout(dropout)(x)
    #x = Dense(int(input_shape/2), activation='relu')(x)
    #x = Dropout(dropout)(x)
    x = Dense(int(input_shape/2), activation='relu')(x)
    x = Dropout(dropout)(x)
    x = Dense(size, activation='sigmoid', name=name+'_output')(x)

    return x


def build_model(levels_size: dict, sequence_size: int = 1280, dropout: float = 0.7) -> tf.keras.models.Model:
    """

    :rtype: tf.keras.models.Model
    """
    input_shape = (sequence_size, 1)
    fcn_size = 1024
    input_layer = Input(shape=input_shape, dtype=tf.float32, name="features")
    x = Dense(fcn_size, activation='relu')(input_layer)
    x = Dropout(dropout)(x)
    x = Dense(int(fcn_size / 2), activation='relu')(x)
    x = Dropout(dropout)(x)
    #x_flat = Flatten()(input_layer)
    
    depth = len(levels_size)

    #x: object = build_cnn(music, input_shape)

    # Construção das camadas sequencialmente
    input = x
    outputs = []
    for level, size in levels_size.items():
        # Construir a camada atual
        current_input = input if level == 'level1' else Concatenate(axis=1)([Flatten()(OutputNormalization()(prev_output)), input_layer])
        output = build_classification(current_input, size, dropout,
                                              input_shape=fcn_size, name=level)

        # Atualizar a saída anterior para a próxima iteração
        prev_output = output
        outputs.append(prev_output)
    

    model = tf.keras.models.Model(inputs=input_layer, outputs=outputs, name="Essentia")

    #     _load_weights(model, weights_path)
    
    optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    model.compile(optimizer=optimizer,
                   loss='binary_crossentropy', metrics=['accuracy']*4)

    return model
'''


'\nfrom keras import layers\nfrom keras.layers import Dense, Dropout, Reshape, Concatenate, Flatten, Input, Normalization\nfrom keras.optimizers import Adam\nimport tensorflow as tf\n\nimport numpy as np\n\n# class OutputNormalization(layers.Layer):\n#     def call(self, x, **kwargs):\n#         norm_output = tf.one_hot(tf.math.argmax(x, axis=1), x.shape[1], dtype=x.dtype)\n#         norm_output = Flatten()(norm_output)\n#         norm_output = Reshape((1,))(norm_output)\n#         return norm_output\n\n#     def compute_output_shape(self, input_shape):\n#         return input_shape\n\nclass OutputNormalization(layers.Layer):\n    def call(self, x, **kwargs):\n        return tf.one_hot(tf.math.argmax(x, axis=1), x.shape[1], dtype=x.dtype)\n\n    def compute_output_shape(self, input_shape):\n        return input_shape\n\n\n\ndef build_cnn(feature, input_shape):\n    x: object = Normalization(input_shape=[input_shape, 1], axis=None)(feature)\n    x = layers.Conv1D(128, 3, activation=\'re

In [10]:
import tensorflow as tf
from keras.layers import Layer, Input, Dense, Dropout, Concatenate, BatchNormalization, Flatten, Reshape
from keras.optimizers import Adam
from keras import backend as K
import numpy as np

class OutputNormalization(Layer):
    def call(self, x):
        return tf.one_hot(tf.math.argmax(x, axis=1), x.shape[1], dtype=x.dtype)

    def compute_output_shape(self, input_shape):
        return input_shape

def build_classification(x, size, dropout, input_shape=1024, name='default'):
    x: object = Dense(input_shape, activation='relu')(x)
    x = Dropout(dropout)(x)
    x = Dense(int(input_shape/2), activation='relu')(x)
    x = Dropout(dropout)(x)
    x = Dense(int(input_shape/4), activation='relu')(x)
    x = Dropout(dropout)(x)
    x = Dense(size, activation='sigmoid', name=name)(x)

    return x


def build_model(levels_size: dict, sequence_size: int = 1280, dropout: float = 0.1) -> tf.keras.models.Model:
    """

    :rtype: tf.keras.models.Model
    """
    input_shape = (sequence_size,)
    music = Input(shape=input_shape, dtype=tf.float32, name="features")
    fcn_size = 1024

    outputs = []
    for level, size in levels_size.items():
        if level != 'level1':
            # Aplicar OutputNormalization na saída anterior
            output_normalized = OutputNormalization()(prev_output)
            #print(output_normalized.shape)
            current_input = Concatenate(axis=1)([output_normalized, music])
        else:
            current_input = music

        current_output = build_classification(current_input, size, dropout, input_shape=fcn_size, name=level)
        print(level)
        # Convert the tensor to a NumPy array
        outputs.append(current_output)
        # Atualizar a saída anterior para a próxima iteração
        prev_output = current_output
        


    model = tf.keras.models.Model(inputs=music, outputs=outputs, name="Essentia")

    #     _load_weights(model, weights_path)
    
    optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy']*4, run_eagerly=True)

    return model

In [11]:
with open(args.metadata_path, 'r') as f:
    metadata = json.loads(f.read())
    print(metadata)

{'sequence_size': 1280, 'max_depth': 4, 'levels_size': [2, 30, 16], 'val_path': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/tfrecords/val', 'train_path': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/tfrecords/train', 'test_path': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/tfrecords/test', 'val_csv': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/val.csv', 'train_csv': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/train.csv', 'test_csv': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/test.csv', 'trainset_count': 16791, 'validationset_count': 2007, 'testset_count': 4814}


In [12]:
metadata['trainset_count']

16791

In [13]:
with open(args.labels_path, 'r') as f:
    labels = json.loads(f.read())

In [14]:
levels_size = {'level1': labels['label_1_count'] ,
                   'level2': labels['label_2_count'] ,
                   'level3': labels['label_3_count'] ,
                   'level4': labels['label_4_count'] }

In [15]:
for level, labels in levels_size.items():
    print(level)

level1
level2
level3
level4


In [16]:
depth = len(levels_size)

In [17]:
depth

4

In [18]:
#ds_train = Dataset(args.trainset_pattern, args.epochs, args.batch_size, levels_size).build(df=False)

In [19]:
#df_train = Dataset(args.trainset_pattern, args.epochs, args.batch_size, levels_size).build(df=True)

In [20]:
from tensorflow.keras.callbacks import EarlyStopping
# from sabotage.model.callback import ValidateCallback, BackupAndRestoreCheckpoint

def run(args: object):
    print(args)

    with open(args.metadata_path, 'r') as f:
        metadata = json.loads(f.read())
        print(metadata)

    with open(args.labels_path, 'r') as f:
        labels = json.loads(f.read())

    levels_size = {'level1': labels['label_1_count'] ,
                   'level2': labels['label_2_count'] ,
                   'level3': labels['label_3_count'] ,
                   'level4': labels['label_4_count'] }

    params: dict = {
        'levels_size': levels_size,
        'sequence_size': metadata['sequence_size'],
        'dropout': args.dropout
    }

    print(params)
    model = build_model(**params)
    print(model.summary())
    
    tf.keras.utils.plot_model(
        model,
        to_file="model.png",
        show_shapes=False,
        show_dtype=False,
        show_layer_names=True,
        rankdir="TB",
        expand_nested=False,
        dpi=96,
        layer_range=None,
        show_layer_activations=False,
        show_trainable=False,
    )
    
    ds_train = Dataset(args.trainset_pattern, args.epochs, args.batch_size, params['levels_size']).build(df=False)
    ds_validation = Dataset(args.valset_pattern, args.epochs, args.batch_size, params['levels_size']).build(df=False)
    callbacks = [EarlyStopping(monitor='loss', patience=args.patience, verbose=1)]
    model.fit(ds_train,
              validation_data=ds_validation,
              steps_per_epoch=metadata['trainset_count'] // args.batch_size,
              validation_steps=metadata['validationset_count'] // args.batch_size,
              epochs=args.epochs,
              callbacks=callbacks)

    model.save(os.path.join(args.model_path, 'best_binary.keras'))


In [21]:

time_start = dt.utcnow()
print("[{}] Experiment started at {}".format(id, time_start.strftime("%H:%M:%S")))
print(".......................................")
print(args)
run(args)
time_end = dt.utcnow()
time_elapsed = time_end - time_start
print(".......................................")
print("[{}] Experiment finished at {} / elapsed time {}s".format(id, time_end.strftime("%H:%M:%S"), time_elapsed.total_seconds()))

[<built-in function id>] Experiment started at 21:00:25
.......................................
batch_size                                                         32
epochs                                                             10
dropout                                                           0.7
patience                                                            1
max_queue_size                                                     64
labels_path         /mnt/disks/data/fma/trains/hierarchical_tworoo...
metadata_path       /mnt/disks/data/fma/trains/hierarchical_tworoo...
trainset_pattern    /mnt/disks/data/fma/trains/hierarchical_tworoo...
testset_pattern     /mnt/disks/data/fma/trains/hierarchical_tworoo...
valset_pattern      /mnt/disks/data/fma/trains/hierarchical_tworoo...
model_path          /mnt/disks/data/fma/trains/hierarchical_tworoo...
dtype: object
batch_size                                                         32
epochs                                            

2024-06-07 18:00:25.331218: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9786 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


level1
level2
level3
level4


Model: "Essentia"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ features            │ (None, 1280)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1024)      │  1,311,744 │ features[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 1024)      │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 512)       │    524,800 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 512)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │    131,328 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 256)       │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ level1 (Dense)      │ (None, 2)         │        514 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_normalizati… │ (None, 2)         │          0 │ level1[0][0]      │
│ (OutputNormalizati… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 1282)      │          0 │ output_normaliza… │
│ (Concatenate)       │                   │            │ features[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1024)      │  1,313,792 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 1024)      │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 512)       │    524,800 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 512)       │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 256)       │    131,328 │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 256)       │          0 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ level2 (Dense)      │ (None, 30)        │      7,710 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_normalizati… │ (None, 30)        │          0 │ level2[0][0]      │
│ (OutputNormalizati… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 1310)      │          0 │ output_normaliza… │
│ (Concatenate)       │                   │            │ features[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 1024)      │  1,342,464 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 1024)      │          0 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 512)       │    524,800 │ dropout_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

...

None


/home/bruno/.cache/pypoetry/virtualenvs/hierarchical_multi-label_classification-GvVd3mwS-py3.11/lib/python3.11/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/10
524/524 ━━━━━━━━━━━━━━━━━━━━ 93s 174ms/step - level1_accuracy: 0.8598 - level2_accuracy: 0.2576 - level3_accuracy: 0.0617 - level4_accuracy: 0.1087 - loss: 0.7239 - val_level1_accuracy: 0.9239 - val_level2_accuracy: 0.4209 - val_level3_accuracy: 0.1290 - val_level4_accuracy: 0.0081 - val_loss: 0.4100
Epoch 2/10
524/524 ━━━━━━━━━━━━━━━━━━━━ 0s 73us/step - level1_accuracy: 1.0000 - level2_accuracy: 0.3182 - level3_accuracy: 0.0909 - level4_accuracy: 0.0000e+00 - loss: 0.3292 - val_level1_accuracy: 0.9130 - val_level2_accuracy: 0.3913 - val_level3_accuracy: 0.2609 - val_level4_accuracy: 0.0000e+00 - val_loss: 0.3801
Epoch 3/10


2024-06-07 18:01:59.414123: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/usr/lib/python3.11/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-06-07 18:01:59.444704: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


524/524 ━━━━━━━━━━━━━━━━━━━━ 91s 174ms/step - level1_accuracy: 0.9178 - level2_accuracy: 0.3846 - level3_accuracy: 0.1055 - level4_accuracy: 0.0124 - loss: 0.4493 - val_level1_accuracy: 0.9269 - val_level2_accuracy: 0.4168 - val_level3_accuracy: 0.1346 - val_level4_accuracy: 0.0081 - val_loss: 0.4000
Epoch 3: early stopping
.......................................
[<built-in function id>] Experiment finished at 21:03:30 / elapsed time 185.709389s
